In [1]:
import sqlalchemy
# import pymysql
import os
import pandas as pd

import sys
sys.path.append('../')
import private_info.db_info as dbi

In [2]:
engine = sqlalchemy.create_engine(f'mysql+pymysql://{dbi.username}:{dbi.password}@{dbi.hostname}:{dbi.port}/{dbi.database}')

In [3]:
db_sqla_conn = engine.connect()

In [4]:
print (db_sqla_conn)

In [5]:
files_list = []

for root, dir, files in os.walk('data'):
    for name in files:
        files_list.append(os.path.join(root,name))

In [6]:
print (files_list)

['data/asset/IWM.csv', 'data/asset/SPY.csv', 'data/asset/QQQ.csv', 'data/gen/fvx.csv', 'data/gen/fed_funds.csv', 'data/gen/tnx.csv', 'data/gen/vix.csv']


In [7]:
for path in files_list:
    
    df = pd.read_csv (path)
    path_parts = path.split('/')
    t_name = path_parts[2][:-4].lower()
    
    print (t_name)
    
    df.to_sql(t_name, engine, if_exists='fail')
    
    engine.execute(f"ALTER TABLE {t_name} DROP COLUMN `Unnamed: 0`;")
    
    engine.execute(f"ALTER TABLE {t_name} CHANGE COLUMN `index` `index` BIGINT NULL AUTO_INCREMENT;")
    
    results = engine.execute(f"SELECT Date FROM {t_name} ORDER BY Date ASC LIMIT 1;")
    results = results.fetchall()[0][0]
    
    engine.execute(f"UPDATE {t_name} SET `index` = 0 WHERE Date = \'{results}\';")
    
    engine.execute(f"ALTER TABLE {t_name} ADD PRIMARY KEY (`index`);")
    
    engine.execute( f"ALTER TABLE {t_name} AUTO_INCREMENT = 0;" )

    

iwm
spy
qqq
fvx
fed_funds
tnx
vix
